<a href="https://colab.research.google.com/github/RusAl84/text_mining/blob/master/25_07_2022_%D0%9C%D0%B0%D1%81%D1%82%D0%B5%D1%80_%D0%BA%D0%BB%D0%B0%D1%81%D1%81_%D0%92%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D1%83_%D0%B5%D1%81%D1%82%D0%B5%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE_%D1%8F%D0%B7%D1%8B%D0%BA%D0%B0_%D0%BD%D0%B0_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Подход TF-IDF

У подхода bag-of-words есть существенный недостаток. Если слово встречается 5 раз в конкретном документе, но и в других документах тоже встречается часто, то его наличие в документе не особо-то о чём-то говорит. Если же слово 5 раз встречается в конкретном документе, но в других документах встречается редко, то его наличие (да ещё и многократное) позволяет хорошо отличать этот документ от других. Однако с точки зрения bag-of-words различий не будет: в обеих ячейках будет просто число 5.

Отчасти это решается исключением стоп-слов (и слишком часто встречающихся слов), но лишь отчасти. Другой идеей является отмасштабировать получившуюся таблицу с учётом "редкости" слова в наборе документов (т.е. с учётом информативности слова).

tfidf=tf∗idf

idf=log((N+1)/(Nw+1))+1

Здесь tf это частота слова в тексте (то же самое, что в bag of words), N - общее число документов, Nw - число документов, содержащих данное слово.

То есть для каждого слова считается отношение общего количества документов к количеству документов, содержащих данное слово (для частых слов оно будет ближе к 1, для редких слов оно будет стремиться к числу, равному количеству документов), и на логарифм от этого числа умножается исходное значение bag-of-words (к числителю и знаменателю прибавляется единичка, чтобы не делить на 0, и к логарифму тоже прибавляется единичка, но это уже технические детали). После этого в sklearn ещё проводится L2-нормализация каждой строки.

В sklearn есть класс для поддержки TF-IDF: TfidfVectorizer, рассмотрим его.

In [ ]:
texts = "\u0422\u043E \u0435\u0441\u0442\u044C \u044D\u0442\u043E  \u043E\u0442\u043A\u0440\u044B\u0432\u0430\u0435\u0442 \u0431\u043E\u043B\u044C\u0448\u0438\u0435 \u043F\u0435\u0440\u0441\u043F\u0435\u043A\u0442\u0438\u0432\u044B. \u0412 PostgreSQL \u0434\u0435\u0439\u0441\u0442\u0432\u0438\u0442\u0435\u043B\u044C\u043D\u043E \u0432\u043F\u043E\u043B\u043D\u0435 \u043C\u043E\u0436\u043D\u043E  \u0441\u0442\u0440\u0438\u043C\u0438\u0442\u044C, \u0435\u0441\u043B\u0438 \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u044C \u043D\u0430\u0448\u0438 \u043E\u043F\u0442\u0438\u043C\u0438\u0437\u0430\u0446\u0438\u0438.  \u0417\u0434\u0435\u0441\u044C \u043F\u043E\u043A\u0430\u0437\u0430\u043D\u043E, \u043A\u0430\u043A \u0440\u0430\u0441\u0442\u0435\u0442 \u0440\u0430\u0437\u043C\u0435\u0440 WAL. \u0421\u043F\u0440\u0430\u0432\u0430 \u043E\u043D \u0437\u0430\u0432\u0438\u0441\u0438\u0442 \u0442\u043E\u043B\u044C\u043A\u043E \u043E\u0442 \u0440\u0430\u0437\u043C\u0435\u0440\u0430 \u0434\u043E\u0431\u0430\u0432\u043B\u044F\u0435\u043C\u043E\u0433\u043E \u043A\u0443\u0441\u043A\u0430, \u0430 \u0441\u043B\u0435\u0432\u0430 \u2014 \u043E\u0442 \u0441\u0430\u043C\u043E\u0439 \u0441\u0442\u0440\u043E\u043A\u0438.   \u041C\u044B \u043F\u043E\u0441\u0447\u0438\u0442\u0430\u043B\u0438 \u0441\u043A\u043E\u0440\u043E\u0441\u0442\u044C (\u041C\u0431/\u0441) \u0434\u043B\u044F \u043E\u0440\u0438\u0433\u0438\u043D\u0430\u043B\u044C\u043D\u043E\u0433\u043E \u043F\u043E\u0441\u0442\u0433\u0440\u0435\u0441\u0430 (\u0441\u043B\u0435\u0432\u0430) \u0438 \u0441 \u043E\u043F\u0442\u0438\u043C\u0438\u0437\u0430\u0446\u0438\u0435\u0439.  \u0421\u043B\u0435\u0432\u0430 \u043C\u044B \u043E\u0433\u0440\u0430\u043D\u0438\u0447\u0438\u043B\u0438\u0441\u044C \u0441\u043B\u0443\u0447\u0430\u0435\u043C, \u043A\u043E\u0433\u0434\u0430 \u043C\u044B \u0434\u043E\u0431\u0430\u0432\u043B\u044F\u0435\u043C  10-\u0431\u0430\u0439\u0442\u043D\u044B\u0435 \u043A\u0443\u0441\u043E\u0447\u043A\u0438, \u0438\u043D\u0430\u0447\u0435 \u0432\u0441\u0451 \u0431\u0443\u0434\u0435\u0442 \u0441\u043B\u0438\u0448\u043A\u043E\u043C \u043C\u0435\u0434\u043B\u0435\u043D\u043D\u043E. \u041C\u044B \u0432\u0438\u0434\u0438\u043C, \u0447\u0442\u043E \u0441\u043A\u043E\u0440\u043E\u0441\u0442\u044C \u043E\u0447\u0435\u043D\u044C \u0431\u044B\u0441\u0442\u0440\u043E \u043F\u0430\u0434\u0430\u0435\u0442 \u0441  1 \u041C\u0431/\u0441 \u0434\u043E 1 \u041A\u0431/\u0441, \u043F\u043E\u043B\u043E\u0441\u0430 \u0441\u043E\u0432\u0441\u0435\u043C \u043C\u0430\u043B\u0435\u043D\u044C\u043A\u0430\u044F. \u0421\u043F\u0440\u0430\u0432\u0430 \u043C\u044B \u0432\u0438\u0434\u0438\u043C, \u0447\u0442\u043E  \u043F\u0440\u043E\u0438\u0437\u0432\u043E\u0434\u0438\u0442\u0435\u043B\u044C\u043D\u043E\u0441\u0442\u044C \u0434\u043E\u043F\u0438\u0441\u044B\u0432\u0430\u043D\u0438\u044F  \u043D\u0435 \u0437\u0430\u0432\u0438\u0441\u0438\u0442 \u043E\u0442 \u0440\u0430\u0437\u043C\u0435\u0440\u0430 \u0434\u0430\u043D\u043D\u044B\u0445 \u0434\u043B\u044F \u043B\u044E\u0431\u044B\u0445 \u0440\u0430\u0437\u043C\u0435\u0440\u043E\u0432 \u0434\u043E\u0431\u0430\u0432\u043B\u0435\u043D\u0438\u0439, \u0438 \u043C\u043E\u0436\u043D\u043E \u043F\u0440\u0435\u0434\u0441\u043A\u0430\u0437\u0430\u0442\u044C, \u0447\u0442\u043E \u0435\u0441\u043B\u0438 \u043C\u044B \u0445\u043E\u0442\u0438\u043C \u0437\u0430\u043D\u044F\u0442\u044C \u043F\u043E\u043B\u043E\u0441\u0443 20 \u041C\u0431/\u0441, \u0442\u043E \u043D\u0430\u0434\u043E \u0430\u043F\u0434\u0435\u0439\u0442\u0438\u0442\u044C \u043A\u0438\u043B\u043E\u0431\u0430\u0439\u0442\u043D\u044B\u043C\u0438 \u0447\u0430\u043D\u043A\u0430\u043C\u0438 \u0438 \u0434\u0435\u0433\u0440\u0430\u0434\u0430\u0446\u0438\u0438 \u043F\u0440\u043E\u0438\u0437\u0432\u043E\u0434\u0438\u0442\u0435\u043B\u044C\u043D\u043E\u0441\u0442\u0438 \u043D\u0435 \u0431\u0443\u0434\u0435\u0442.   \u0412\u043C\u0435\u0441\u0442\u043E \u0417\u0430\u043A\u043B\u044E\u0447\u0435\u043D\u0438\u044F \u0412 \u044D\u0442\u043E\u0439 \u0441\u0435\u0440\u0438\u0438 \u0441\u0442\u0430\u0442\u0435\u0439 \u044F \u0440\u0430\u0441\u0441\u043A\u0430\u0437\u0430\u043B \u043F\u0440\u043E \u0432\u043E\u0437\u043C\u043E\u0436\u043D\u043E\u0441\u0442\u0438 \u0443\u043B\u0443\u0447\u0448\u0435\u043D\u0438\u044F PostgreSQL \u0434\u043B\u044F \u044D\u0444\u0444\u0435\u043A\u0442\u0438\u0432\u043D\u043E\u0433\u043E \u0445\u0440\u0430\u043D\u0435\u043D\u0438\u044F \u0431\u043E\u043B\u044C\u0448\u0438\u0445 \u0437\u043D\u0430\u0447\u0435\u043D\u0438\u0439 \u043D\u0430 \u043F\u0440\u0438\u043C\u0435\u0440\u0435 \u043F\u043E\u043F\u0443\u043B\u044F\u0440\u043D\u043E\u0433\u043E \u0442\u0438\u043F\u0430 \u0434\u0430\u043D\u043D\u044B\u0445 JSONB \u0438 \u0431\u044B\u0441\u0442\u0440\u043E\u0433\u043E \u0434\u043E\u043F\u0438\u0441\u044B\u0432\u0430\u043D\u0438\u044F \u0431\u0438\u043D\u0430\u0440\u043D\u044B\u0445 \u0434\u0430\u043D\u043D\u044B\u0445. PostgreSQL \u0441\u043B\u0430\u0432\u0438\u0442\u0441\u044F \u0441\u0432\u043E\u0435\u0439 \u0440\u0430\u0441\u0448\u0438\u0440\u044F\u0435\u043C\u043E\u0441\u0442\u044C\u044E, \u043F\u043E\u044D\u0442\u043E\u043C\u0443 \u043B\u043E\u0433\u0438\u0447\u043D\u043E \u0435\u0435 \u0440\u0430\u0441\u0448\u0438\u0440\u0438\u0442\u044C \u0438 \u043D\u0430 TOAST \u2014 \u0442\u0430\u043A \u0447\u0442\u043E\u0431\u044B \u0445\u0440\u0430\u043D\u0435\u043D\u0438\u0435 \u0431\u043E\u043B\u044C\u0448\u0438\u0445 \u0437\u043D\u0430\u0447\u0435\u043D\u0438\u0439 \u0431\u044B\u043B\u043E datatype aware.  \u041C\u044B \u043F\u0440\u0435\u0434\u043B\u043E\u0436\u0438\u043B\u0438 \u0441\u0435\u0440\u0438\u044E \u043F\u0430\u0442\u0447\u0435\u0439, \u0440\u0435\u0430\u043B\u0438\u0437\u0443\u044E\u0449\u0438\u0445 API \u0434\u043B\u044F TOAST (\u0441\u043C. Pluggable TOAST), \u043D\u0430 \u043E\u0441\u043D\u043E\u0432\u0435 \u043A\u043E\u0442\u043E\u0440\u043E\u0433\u043E \u043C\u043E\u0436\u043D\u043E \u0440\u0430\u0437\u0440\u0430\u0431\u0430\u0442\u044B\u0432\u0430\u0442\u044C TOAST, \u043E\u043F\u0442\u0438\u043C\u0438\u0437\u0438\u0440\u043E\u0432\u0430\u043D\u043D\u044B\u0439 \u0434\u043B\u044F \u043E\u043F\u0440\u0435\u0434\u0435\u043B\u0435\u043D\u043D\u043E\u0433\u043E \u0442\u0438\u043F\u0430 \u0434\u0430\u043D\u043D\u044B\u0445. \u041D\u0430\u043F\u0440\u0438\u043C\u0435\u0440, \u0432\u0441\u0435 \u043E\u043F\u0438\u0441\u0430\u043D\u043D\u044B\u0435 \u043E\u043F\u0442\u0438\u043C\u0438\u0437\u0430\u0446\u0438\u0438 \u0434\u043B\u044F JSONB \u043C\u043E\u0436\u043D\u043E \u0440\u0435\u0430\u043B\u0438\u0437\u043E\u0432\u0430\u0442\u044C \u0432 \u0432\u0438\u0434\u0435 \u0440\u0430\u0441\u0448\u0438\u0440\u0435\u043D\u0438\u044F. \u041D\u0430\u0434\u0435\u0435\u043C\u0441\u044F \u0437\u0430\u043A\u043E\u043C\u043C\u0438\u0442\u0438\u0442\u044C \u0432\u0441\u0435 \u044D\u0442\u043E \u0434\u043B\u044F \u0441\u043B\u0435\u0434\u0443\u044E\u0449\u0435\u0439 \u0432\u0435\u0440\u0441\u0438\u0438 PG15.  \u0412\u0438\u0434\u0435\u043E \u043C\u043E\u0435\u0433\u043E \u0432\u044B\u0441\u0442\u0443\u043F\u043B\u0435\u043D\u0438\u044F \u043D\u0430 Saint HighLoad++ 2021:   \u041A\u043E\u043D\u0444\u0435\u0440\u0435\u043D\u0446\u0438\u044F HighLoad++ Foundation 2022 \u043F\u0440\u043E\u0439\u0434\u0435\u0442 17 \u0438 18 \u043C\u0430\u0440\u0442\u0430 \u0432 \u041C\u043E\u0441\u043A\u0432\u0435 \u0432 \u041A\u0440\u043E\u043A\u0443\u0441 \u042D\u043A\u0441\u043F\u043E. \u2014 1 \u0444\u0435\u0432\u0440\u0430\u043B\u044F. \u041F\u043B\u0430\u043D\u0438\u0440\u0443\u0439\u0442\u0435 \u0441\u0432\u043E\u0435 \u0443\u0447\u0430\u0441\u0442\u0438\u0435, \u0440\u0430\u0441\u043F\u0438\u0441\u0430\u043D\u0438\u0435 \u0438 \u043F\u043E\u043B\u043D\u044B\u0439 \u0441\u043F\u0438\u0441\u043E\u043A \u0442\u0435\u043C \u0441 \u0442\u0435\u0437\u0438\u0441\u0430\u043C\u0438 \u0443\u0436\u0435 \u043D\u0430 \u0441\u0430\u0439\u0442\u0435.  \u041D\u0430\u0448 \u0434\u043E\u043A\u043B\u0430\u0434 \u0441 \u043A\u043E\u043B\u043B\u0435\u0433\u0430\u043C\u0438 \u2014 Pluggable TOAST or One TOAST fits ALL \u2014 \u0431\u0443\u0434\u0435\u0442 \u043B\u043E\u0433\u0438\u0447\u043D\u044B\u043C \u043F\u0440\u043E\u0434\u043E\u043B\u0436\u0435\u043D\u0438\u0435\u043C \u0442\u043E\u0433\u043E, \u0447\u0442\u043E \u044F \u0440\u0430\u0441\u0441\u043A\u0430\u0437\u0430\u043B \u0432 \u044D\u0442\u043E\u0439 \u0441\u0435\u0440\u0438\u0438 \u0441\u0442\u0430\u0442\u0435\u0439. \u0411\u0438\u043B\u0435\u0442\u044B \u043C\u043E\u0436\u043D\u043E \u043A\u0443\u043F\u0438\u0442\u044C \u0437\u0434\u0435\u0441\u044C.  \u0414\u043E \u0432\u0441\u0442\u0440\u0435\u0447\u0438 \u043D\u0430 \u043A\u043E\u043D\u0444\u0435\u0440\u0435\u043D\u0446\u0438\u0438!  \u0422\u0435\u0433\u0438: postgresqlpostgresjsonjsonbstreamingstream processinghigh performancehighload++postgresql performancetoast \u0425\u0430\u0431\u044B: \u0411\u043B\u043E\u0433 \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u0438 \u041A\u043E\u043D\u0444\u0435\u0440\u0435\u043D\u0446\u0438\u0438 \u041E\u043B\u0435\u0433\u0430 \u0411\u0443\u043D\u0438\u043D\u0430 (\u041E\u043D\u0442\u0438\u043A\u043E)\u0412\u044B\u0441\u043E\u043A\u0430\u044F \u043F\u0440\u043E\u0438\u0437\u0432\u043E\u0434\u0438\u0442\u0435\u043B\u044C\u043D\u043E\u0441\u0442\u044CPostgreSQL\u0410\u0434\u043C\u0438\u043D\u0438\u0441\u0442\u0440\u0438\u0440\u043E\u0432\u0430\u043D\u0438\u0435 \u0431\u0430\u0437 \u0434\u0430\u043D\u043D\u044B\u0445\u0425\u0440\u0430\u043D\u0438\u043B\u0438\u0449\u0430 \u0434\u0430\u043D\u043D\u044B\u0445 +33  58 1   \u041A\u043E\u043D\u0444\u0435\u0440\u0435\u043D\u0446\u0438\u0438 \u041E\u043B\u0435\u0433\u0430 \u0411\u0443\u043D\u0438\u043D\u0430 (\u041E\u043D\u0442\u0438\u043A\u043E) \u041F\u0440\u043E\u0444\u0435\u0441\u0441\u0438\u043E\u043D\u0430\u043B\u044C\u043D\u044B\u0435 \u043A\u043E\u043D\u0444\u0435\u0440\u0435\u043D\u0446\u0438\u0438 \u0434\u043B\u044F IT-\u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A\u043E\u0432 \xD7\u041F\u043E\u0434\u043F\u0438\u0441\u0430\u0442\u044C\u0441\u044F FacebookFacebookTwitterTelegram  66 \u041A\u0430\u0440\u043C\u0430 0 \u0420\u0435\u0439\u0442\u0438\u043D\u0433 Oleg Bartunov @zen \u041F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u044C  \xD7\u041F\u043E\u0434\u043F\u0438\u0441\u0430\u0442\u044C\u0441\u044F \u041A\u043E\u043C\u043C\u0435\u043D\u0442\u0430\u0440\u0438\u0438 1    vtikunov 23.02.2022 \u0432 12:51 \u0418\u043D\u0442\u0435\u0440\u0435\u0441\u043D\u043E, \u0447\u0442\u043E \u0437\u0430 \u043A\u0435\u0439\u0441 \u0442\u0430\u043A\u043E\u0439 \u0441\u043E 100\u041C\u0431 \u0432 \u043E\u0434\u043D\u043E\u043C \u043F\u043E\u043B\u0435. \u041A\u0430\u0436\u0435\u0442\u0441\u044F \u043B\u043E\u0433\u0438\u0447\u043D\u044B\u043C, \u0447\u0442\u043E postgres \u0438\u0437 \u043A\u043E\u0440\u043E\u0431\u043A\u0438 \u043D\u0435 \u043D\u043E\u0440\u043C\u0430\u043B\u044C\u043D\u043E \u0440\u0430\u0431\u043E\u0442\u0430\u0435\u0442 \u0441 \u0442\u0430\u043A\u0438\u043C \u0440\u0430\u0437\u043C\u0435\u0440\u043E\u043C.  0 \u0412\u044B \u043C\u043E\u0436\u0435\u0442\u0435 \u043E\u0441\u0442\u0430\u0432\u043B\u044F\u0442\u044C \u043A\u043E\u043C\u043C\u0435\u043D\u0442\u0430\u0440\u0438\u0438 \u0442\u043E\u043B\u044C\u043A\u043E \u043A \u0441\u0432\u0435\u0436\u0438\u043C \u043F\u043E\u0441\u0442\u0430\u043C \u041F\u041E\u0425\u041E\u0416\u0418\u0415 \u041F\u0423\u0411\u041B\u0418\u041A\u0410\u0426\u0418\u0418 28 \u0430\u043F\u0440\u0435\u043B\u044F \u0432 13:49 \u0421\u0432\u043E\u0431\u043E\u0434\u043D\u044B\u0439 \u0434\u043E\u0441\u0442\u0443\u043F \u043A \u0442\u0440\u0430\u043D\u0441\u043B\u044F\u0446\u0438\u0438 \u0438\u0437 \xAB\u0422\u0435\u0440\u043C\u0438\u043D\u0443\u0441\u0430\xBB, \u0433\u043B\u0430\u0432\u043D\u043E\u0433\u043E \u0437\u0430\u043B\u0430 Highload++ Foundation 2022 +7 513  3 0 24 \u044F\u043D\u0432\u0430\u0440\u044F \u0432 00:26 \u0411\u043E\u0440\u044C\u0431\u0430 \u0441  TOAST \u0438\u043B\u0438 \u0431\u0443\u0434\u0443\u0449\u0435\u0435 JSONB \u0432 PostgreSQL +56 13K  71 8 27 \u0434\u0435\u043A\u0430\u0431\u0440\u044F 2021 \u0432 11:10 \u041F\u0440\u043E\u043A\u043B\u044F\u0442\u044C\u0435 TOAST \u0438 \u0441 \u043A\u0430\u043A\u0438\u043C \u043C\u0430\u0441\u043B\u043E\u043C \u0435\u0433\u043E \u0435\u0441\u0442 JSONB +25 6.9K  28 0 \u041B\u0423\u0427\u0428\u0418\u0415 \u041F\u0423\u0411\u041B\u0418\u041A\u0410\u0426\u0418\u0418 \u0417\u0410 \u0421\u0423\u0422\u041A\u0418 \u0441\u0435\u0433\u043E\u0434\u043D\u044F \u0432 09:22 \u0412\u0442\u043E\u0440\u0430\u044F \u0436\u0438\u0437\u043D\u044C \u043A\u0438\u0442\u0430\u0439\u0441\u043A\u043E\u0433\u043E \u0447\u0443\u0434\u043E-\u0448\u043D\u0443\u0440\u043A\u0430 J2534 +40 6.8K  15 10 +10" #@param {type:"string"}
stexts = texts
print(texts)

То есть это  открывает большие перспективы. В PostgreSQL действительно вполне можно  стримить, если использовать наши оптимизации.  Здесь показано, как растет размер WAL. Справа он зависит только от размера добавляемого куска, а слева — от самой строки.   Мы посчитали скорость (Мб/с) для оригинального постгреса (слева) и с оптимизацией.  Слева мы ограничились случаем, когда мы добавляем  10-байтные кусочки, иначе всё будет слишком медленно. Мы видим, что скорость очень быстро падает с  1 Мб/с до 1 Кб/с, полоса совсем маленькая. Справа мы видим, что  производительность дописывания  не зависит от размера данных для любых размеров добавлений, и можно предсказать, что если мы хотим занять полосу 20 Мб/с, то надо апдейтить килобайтными чанками и деградации производительности не будет.   Вместо Заключения В этой серии статей я рассказал про возможности улучшения PostgreSQL для эффективного хранения больших значений на примере популярного типа данных JSONB и быстрого дописывания бинарных данн

In [ ]:
# !pip install scikit-learn
!pip install pandas==1.2.3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def printTF_IDF(texts):
  records_count = 30
  tfIdfTransformer = TfidfVectorizer(ngram_range=(1, 4), use_idf=True, max_features=records_count)
  countVectorizer = CountVectorizer(ngram_range=(1, 4), max_features=records_count)
  wordCount = countVectorizer.fit_transform([texts])
  TfIdf = tfIdfTransformer.fit_transform([texts])
  names = countVectorizer.get_feature_names()
  df=[]
  df = pd.DataFrame(list(names), columns=['names'])
  df = df.assign(Word_Count=wordCount.T.todense())
  df = df.assign(TF_IDF=TfIdf.T.todense())
  df = df.sort_values('TF_IDF', ascending=False)
  print(df)

In [ ]:
printTF_IDF(texts)

                  names  Word_Count    TF_IDF
6                 toast           8  0.364013
9                   для           8  0.364013
16                   мы           7  0.318511
29                  что           6  0.273009
19                   на           6  0.273009
8                данных           5  0.227508
15                можно           5  0.227508
5            postgresql           5  0.227508
0                    10           4  0.182006
4                 jsonb           4  0.182006
13          конференции           4  0.182006
3              highload           3  0.136505
28                слева           3  0.136505
25                   от           3  0.136505
20                   не           3  0.136505
7                 будет           3  0.136505
1                  2022           3  0.136505
14                   мб           3  0.136505
17             мы видим           2  0.091003
2                    28           2  0.091003
12                   из           

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


удалим знаки переноса строк и сделаем текст в нижнем регистре

In [ ]:
  str2 = ''
  for item in texts.split():
    str2 = str2 + ' ' + item
  texts = str(texts)
  texts = texts.lower()
  # print(texts)
  texts = texts.replace('\n', ' ')
  print(texts)

то есть это  открывает большие перспективы. в postgresql действительно вполне можно  стримить, если использовать наши оптимизации.  здесь показано, как растет размер wal. справа он зависит только от размера добавляемого куска, а слева — от самой строки.   мы посчитали скорость (мб/с) для оригинального постгреса (слева) и с оптимизацией.  слева мы ограничились случаем, когда мы добавляем  10-байтные кусочки, иначе всё будет слишком медленно. мы видим, что скорость очень быстро падает с  1 мб/с до 1 кб/с, полоса совсем маленькая. справа мы видим, что  производительность дописывания  не зависит от размера данных для любых размеров добавлений, и можно предсказать, что если мы хотим занять полосу 20 мб/с, то надо апдейтить килобайтными чанками и деградации производительности не будет.   вместо заключения в этой серии статей я рассказал про возможности улучшения postgresql для эффективного хранения больших значений на примере популярного типа данных jsonb и быстрого дописывания бинарных данн

удалим цифры

In [ ]:
  str2 = ''
  for c in texts:
      if c not in ('0', "1", '2', '3', '4', '5', '6', '7', '8', '9', '«', '»', '–', "\""):
          str2 = str2 + c
  texts = str2
  str2 = ''
  print(texts)

то есть это  открывает большие перспективы. в postgresql действительно вполне можно  стримить, если использовать наши оптимизации.  здесь показано, как растет размер wal. справа он зависит только от размера добавляемого куска, а слева — от самой строки.   мы посчитали скорость (мб/с) для оригинального постгреса (слева) и с оптимизацией.  слева мы ограничились случаем, когда мы добавляем  -байтные кусочки, иначе всё будет слишком медленно. мы видим, что скорость очень быстро падает с   мб/с до  кб/с, полоса совсем маленькая. справа мы видим, что  производительность дописывания  не зависит от размера данных для любых размеров добавлений, и можно предсказать, что если мы хотим занять полосу  мб/с, то надо апдейтить килобайтными чанками и деградации производительности не будет.   вместо заключения в этой серии статей я рассказал про возможности улучшения postgresql для эффективного хранения больших значений на примере популярного типа данных jsonb и быстрого дописывания бинарных данных. po

удалим знаки пунктуации

In [ ]:
import string
pattern = string.punctuation
for c in texts:
    if c not in pattern:
        str2 = str2 + c
    else:
        str2 = str2 + " "
texts = str2
str2 = ''
print(texts)

то есть это  открывает большие перспективы  в postgresql действительно вполне можно  стримить  если использовать наши оптимизации   здесь показано  как растет размер wal  справа он зависит только от размера добавляемого куска  а слева — от самой строки    мы посчитали скорость  мб с  для оригинального постгреса  слева  и с оптимизацией   слева мы ограничились случаем  когда мы добавляем   байтные кусочки  иначе всё будет слишком медленно  мы видим  что скорость очень быстро падает с   мб с до  кб с  полоса совсем маленькая  справа мы видим  что  производительность дописывания  не зависит от размера данных для любых размеров добавлений  и можно предсказать  что если мы хотим занять полосу  мб с  то надо апдейтить килобайтными чанками и деградации производительности не будет    вместо заключения в этой серии статей я рассказал про возможности улучшения postgresql для эффективного хранения больших значений на примере популярного типа данных jsonb и быстрого дописывания бинарных данных  po

загрузим стоп слова, используем библиотеку NLTK

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Добавить свои стоп-слова:

In [ ]:
my_stop_words = "\u0420\u0443\u0441\u0430\u043A\u043E\u0432 \u041C\u0438\u0445\u0430\u043B\u044B\u0447" #@param {type:"string"}
print(my_stop_words)

Русаков Михалыч


Удалим стоп слова из текста

In [ ]:
  from nltk.corpus import stopwords
  russian_stopwords = stopwords.words("russian")

  for word in my_stop_words.split():
    russian_stopwords.append(word)
  str2 = ''
  for word in texts.split():
      if word not in (russian_stopwords):
          str2 = str2 + " " + word
  texts = str2
  str2 = ''
  for word in texts.split():
      if len(word) > 1:
          str2 = str2 + " " + word
  texts = str2
  print(russian_stopwords)

  

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

рассчитаем TF-IDF для отфильтрованных данных

In [ ]:
printTF_IDF(texts)

                              names  Word_Count    TF_IDF
6                             toast           8  0.488678
9                            данных           5  0.305424
5                        postgresql           5  0.305424
19                               мб           4  0.244339
2                             jsonb           4  0.244339
14                      конференции           4  0.244339
0                          highload           3  0.183254
25                            слева           3  0.183254
26                           справа           2  0.122169
18                         логичным           2  0.122169
27                           статей           2  0.122169
24                      оптимизации           2  0.122169
23                           онтико           2  0.122169
22              олега бунина онтико           2  0.122169
21                     олега бунина           2  0.122169
28                             типа           2  0.122169
20            

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


#Лемматизация
Лемматизация (англ. lemmatization) - это метод морфологического анализа, который сводится к приведению словоформы к ее первоначальной словарной форме (лемме).

Метод лемматизации применяется в поисковых алгоритмах в процессе схематизации веб-документов при их индексировании.

В результате лемматизации от словоформы отбрасываются флективные окончания и возвращается основная или словарная форма слова.

Например, в русском языке словарной формой считается:

существительные - именительный падеж, единственное число (мечами - меч)
глаголы - инфинитивная форма (читали - читать)
прилагательные - единственное число, именительный падеж, мужской род (заснеженными - заснеженный)

In [ ]:
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
ltexts = ""
print(texts)
for word in texts.split():
    if len(str(word)) > 2:
        ltexts+=" " + morph.parse(word)[0].normal_form
print(ltexts)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 это открывает большие перспективы postgresql действительно вполне стримить использовать наши оптимизации показано растет размер wal справа зависит размера добавляемого куска слева самой строки посчитали скорость мб оригинального постгреса слева оптимизацией слева ограничились случаем добавляем байтные кусочки иначе всё слишком медленно видим скорость очень быстро падает мб кб полоса маленькая справа видим производительность дописывания зависит размера данных любых размеров добавлений предсказать хотим занять полосу мб апдейтить килобайтными чанками деградации производительности вместо заключения серии статей рассказал возможности улучшения postgresql эффективного хранения больших значений примере популярного типа данных jsonb быстрого дописывания бинарных данных postgresql славится своей расширяемостью поэтому логично расширить toast хранение больших значений datatype aware предложили се

/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.g

рассчитаем TF-IDF для Лемматизированных данных

In [ ]:
printTF_IDF(ltexts)

                  names  Word_Count    TF_IDF
4                 toast           8  0.461112
19               размер           5  0.288195
3            postgresql           5  0.288195
7                данные           5  0.288195
11          конференция           5  0.288195
2                 jsonb           4  0.230556
0              highload           3  0.172917
23                серия           3  0.172917
24                слева           3  0.172917
17          оптимизация           3  0.172917
5               больший           3  0.172917
25               справа           2  0.115278
27                  тип           2  0.115278
28           тип данные           2  0.115278
22                 свой           2  0.115278
21          реализовать           2  0.115278
20           рассказать           2  0.115278
26                стать           2  0.115278
15    олег бунин онтико           2  0.115278
18   производительность           2  0.115278
16               онтико           

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


#Суммаризации текстов 

Задача суммаризации текстов (автореферирование) - одна из ключевых, широко обсуждаемых задач NLP. Она состоит в сжатии больших объемов текста до связного краткого содержания, отражающего только основные идеи.


#Алгоритм TextRank
TextRank - это алгоритм, основанный на PageRank, который часто используется для извлечения ключевых слов и суммирования текста. 
PageRank (PR) - это алгоритм, используемый для расчета веса веб-страниц. 


In [ ]:
from gensim.summarization import summarize, keywords
print("Исходный текст:")
print(stexts)
print("\nРезультат работы TextRank:")
print(summarize(str(stexts)))

Исходный текст:
То есть это  открывает большие перспективы. В PostgreSQL действительно вполне можно  стримить, если использовать наши оптимизации.  Здесь показано, как растет размер WAL. Справа он зависит только от размера добавляемого куска, а слева — от самой строки.   Мы посчитали скорость (Мб/с) для оригинального постгреса (слева) и с оптимизацией.  Слева мы ограничились случаем, когда мы добавляем  10-байтные кусочки, иначе всё будет слишком медленно. Мы видим, что скорость очень быстро падает с  1 Мб/с до 1 Кб/с, полоса совсем маленькая. Справа мы видим, что  производительность дописывания  не зависит от размера данных для любых размеров добавлений, и можно предсказать, что если мы хотим занять полосу 20 Мб/с, то надо апдейтить килобайтными чанками и деградации производительности не будет.   Вместо Заключения В этой серии статей я рассказал про возможности улучшения PostgreSQL для эффективного хранения больших значений на примере популярного типа данных JSONB и быстрого дописыван

#Алгоритм Rake
RAKE: Rapid Automatic Keyword Extraction Algorithm
Алгоритм RAKE извлекает ключевые слова с помощью основанного на разделителе подхода, чтобы идентифицировать ключевые слова кандидата и баллы их использующий совместные встречаемости слова, которые появляются в ключевых словах кандидата. Ключевые слова могут содержать несколько лексем. Кроме того, алгоритм RAKE также объединяет ключевые слова, когда они кажутся многократно, разделенными тем же разделителем слияния.


In [ ]:
!pip install rake_nltk
from rake_nltk import Metric, Rake
r = Rake(language="russian")
import nltk
nltk.download('punkt')
r.extract_keywords_from_text(stexts)
mas = r.get_ranked_phrases()
set2 = set()
for item in mas:
    if not "nan" in str(item).replace(" nan ", " "):
        set2.add(str(item).replace(" nan ", " "))
mas = list(set2)
print("Исходный текст:")
print(stexts)
print("\nРезультат работы TextRammk:")
for item in mas:
  print(item)
# print(str(mas))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Исходный текст:
То есть это  открывает большие перспективы. В PostgreSQL действительно вполне можно  стримить, если использовать наши оптимизации.  Здесь показано, как растет размер WAL. Справа он зависит только от размера добавляемого куска, а слева — от самой строки.   Мы посчитали скорость (Мб/с) для оригинального постгреса (слева) и с оптимизацией.  Слева мы ограничились случаем, когда мы добавляем  10-байтные кусочки, иначе всё будет слишком медленно. Мы видим, что скорость очень быстро падает с  1 Мб/с до 1 Кб/с, полоса совсем маленькая. Справа мы видим, что  производительность дописывания  не зависит от размера данных для любых размеров добавлений, и можно предсказать, что если мы хотим занять полосу 20 Мб/с, то надо апдейтить килобайтными чанками и деградации производительности не будет.   Вместо Заключения В этой серии статей я рассказал про возможности улучшения PostgreSQL для э

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#Тематическое моделирование
Тематическое моделирование — это метод извлечения тем из текста. Latent Dirichlet Allocation (LDA) — популярный алгоритм моделирования тем реализованные в том числе в пакете Gensim. Основная задача алгоритмов ТМ, заключается в том что бы полученные темы были хорошего качество, понятными, самозначимыми и разделенными. Достижение этих целей во многом зависит от качества предварительной обработки текста и стратегии поиска оптимального количества тем. 

##Импорт пакетов


In [ ]:
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Подход LDA к тематическому моделированию заключается в том, что каждый документ рассматривается как набор тем в определенной пропорции. И каждая тема как набор ключевых слов, опять же, в определенной пропорции.

После того, как вы укажете алгоритму количество тем, все, что он сделает, — это отобразит распределение тем в документах и распределение ключевых слов по темам.

Тема — это не что иное, как набор доминирующих ключевых слов. Просто взглянув на ключевые слова, вы сможете определить, о чем эта тема.

Ниже приведены ключевые факторы для получения хороших разделительных тем:

Качество обработки текста.
Разнообразие тем, о которых говорится в тексте.
Выбор алгоритма моделирование тем.
Количество тем, указанных в алгоритме.
Алгоритмы настройки параметров.

# Подготовим стоп-слова


In [ ]:
from nltk.corpus import stopwords
stop_words = russian_stopwords

##Лемматизация
Лемматизация — это не что иное, как преобразование слова в его корневое слово. Например: лемма слова «machines» — это «machine». Аналогично, «walking» -> «walk», «mice» -> «mouse» и так далее.

In [ ]:
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
ltexts = ""
print(texts)
for word in texts.split():
    if len(str(word)) > 2:
        ltexts+=" " + morph.parse(word)[0].normal_form
print(ltexts)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 это открывает большие перспективы postgresql действительно вполне стримить использовать наши оптимизации показано растет размер wal справа зависит размера добавляемого куска слева самой строки посчитали скорость мб оригинального постгреса слева оптимизацией слева ограничились случаем добавляем байтные кусочки иначе всё слишком медленно видим скорость очень быстро падает мб кб полоса маленькая справа видим производительность дописывания зависит размера данных любых размеров добавлений предсказать хотим занять полосу мб апдейтить килобайтными чанками деградации производительности вместо заключения серии статей рассказал возможности улучшения postgresql эффективного хранения больших значений примере популярного типа данных jsonb быстрого дописывания бинарных данных postgresql славится своей расширяемостью поэтому логично расширить toast хранение больших значений datatype aware предложили се

##Создадим словарь и корпус.
Двумя основными входными данными для тематической модели LDA являются словарь (id2word) и корпус. Давайте создадим их.

In [ ]:
from nltk.util import ngrams
def make_bigrams(ttexts):
    texts = ' '.join(ttexts)
    token = nltk.word_tokenize(ttexts)
    bigrams = list(ngrams(token, 2))
    # print(bigrams)
    return bigrams
def make_trigrams(ttexts):
    texts = ' '.join(ttexts)
    token = nltk.word_tokenize(ttexts)
    trigrams = list(ngrams(token, 2))
    return trigrams

btexts = make_bigrams(texts) + make_trigrams(texts)
id2word = corpora.Dictionary(btexts)
corpus = [id2word.doc2bow(text) for text in btexts]
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('открывает', 1), ('это', 1)]]

Или вы можете увидеть удобочитаемую форму самого корпуса.

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('открывает', 1), ('это', 1)]]

##Построим тематическую модель
У нас есть все необходимое для обучения модели LDA. В дополнение к корпусу и словарю необходимо также указать количество тем.

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3,                          # Количество тем (отдельная тема для диссертации)
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

##Просмотр тем в модели LDA
Вышеупомянутая модель LDA построена на 20 различных темах, где каждая тема представляет собой комбинацию ключевых слов, и каждое ключевое слово вносит определенный вес в тему.

Вы можете увидеть ключевые слова для каждой темы и вес (важность) каждого ключевого слова, используя lda_model.print_topics().

In [ ]:
lda_model.print_topics()

[(0,
  '0.102*"toast" + 0.028*"конференции" + 0.028*"foundation" + 0.020*"highload" + 0.017*"борьба" + 0.017*"января" + 0.017*"каким" + 0.017*"проклятье" + 0.017*"будущее" + 0.015*"×подписаться"'),
 (1,
  '0.114*"postgresql" + 0.060*"данных" + 0.036*"декабря" + 0.023*"значений" + 0.023*"видим" + 0.023*"больших" + 0.021*"справа" + 0.021*"размера" + 0.021*"скорость" + 0.021*"зависит"'),
 (2,
  '0.071*"jsonb" + 0.038*"публикации" + 0.031*"сутки" + 0.031*"ест" + 0.031*"китайского" + 0.031*"чудо" + 0.031*"сегодня" + 0.031*"лучшие" + 0.031*"вторая" + 0.031*"жизнь"')]

##Визуализация темы и ключевых слов
Теперь, когда модель LDA создана, следующим шагом является изучение созданных тем и связанных с ними ключевых слов. Нет лучшего инструмента, чем интерактивная диаграмма пакета pyLDAvis.

In [ ]:
!pip install pyLDAvis==2.1.2
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.172455 -0.134411       1        1  44.419254
2     -0.219469 -0.063271       2        1  33.125148
1      0.047014  0.197682       3        1  22.455597, topic_info=           Term       Freq      Total Category  logprob  loglift
4    postgresql  27.000000  27.000000  Default  30.0000  30.0000
83        toast  47.000000  47.000000  Default  29.0000  29.0000
74        jsonb  25.000000  25.000000  Default  28.0000  28.0000
47       данных  14.000000  14.000000  Default  27.0000  27.0000
187  публикации  13.000000  13.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
198     декабря   8.341245  14.096230   Topic3  -3.3360   0.9689
20        слева   3.402987   5.937651   Topic3  -4.2325   0.9370
37     медленно   1.586101   2.786178   Topic3  -4.9959   0.9302
73         типа   2.965351   5.990049   Topic3  -4.3702   0.7905
25           мб   3.862203   9.121221   Topic3  -4.1059   0.6343

[145 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
136       1  0.984332           all
166       2  0.911629      bartunov
135       1  0.984332          fits
115       1  0.969786    foundation
113       1  0.908879      highload
...     ...       ...           ...
120       1  0.984332         экспо
1         2  0.900314           это
67        3  0.889732  эффективного
195       1  0.585245        января
195       2  0.365778        января

[118 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])